<a href="https://colab.research.google.com/github/ajithpious/Colab-Notebooks/blob/main/LV_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Libraries

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!cp /content/drive/MyDrive/MscProj/Code/net.py /content/

#Data Preparation

## Loading from TFRecords

In [4]:
from tensorflow.python.ops.math_ops import reduce_min
from tensorflow.python.ops.ragged.ragged_math_ops import reduce_max

# fig,ax=plt.subplots(nrows=1,ncols=2)
rawdataset=tf.data.TFRecordDataset("/content/drive/MyDrive/Data/MscProj/Data/tfRecord/data.tfrecords")
# print(rawdataset)
featureDescription={"image":tf.io.FixedLenFeature([],tf.string,default_value=''),"mask":tf.io.FixedLenFeature([],tf.string,default_value='')}
def scale(tensor):
  # tensor = tf.div(tf.subtract(tensor, tf.reduce_min(tensor)), 
  #  tf.subtract(tf.reduce_max(tensor),tf.reduce_min(tensor)))
  tensor=(tensor-tf.reduce_min(tensor))/(tf.reduce_max(tensor)-tf.reduce_min(tensor))
  return tensor
def _parse_function(example_proto):
  features=tf.io.parse_single_example(example_proto, featureDescription)
  image=tf.io.decode_raw(features['image'],tf.int32)
  mask=tf.io.decode_raw(features['mask'],tf.uint8)
  mask=tf.cast(mask,tf.float32)
  image=tf.cast(image,tf.float32)
  print(mask)
  image=tf.reshape(image,(256,256,1))
  mask=tf.reshape(mask,(256,256,1))
  image=scale(image)
  mask=scale(mask)
  return image,mask
parsed_dataset=rawdataset.map(_parse_function).shuffle(100)
# map(_parse_image)


Tensor("Cast:0", shape=(None,), dtype=float32)


In [5]:
#Spliting the dataset
l=0
for i in parsed_dataset:
  l=l+1
trainSize=int(0.7*l)
valSize=int(0.15*l)
testSize=int(0.15*l)
train_dataset=parsed_dataset.take(trainSize)
test_dataset=parsed_dataset.skip(trainSize)
val_dataset=test_dataset.take(valSize)
test_dataset=test_dataset.skip(testSize)

In [6]:
#Creating batches 
train_batches = (
    train_dataset
    .cache()
    .shuffle(1000)
    .batch(64)
    .prefetch(buffer_size=tf.data.AUTOTUNE))
val_batches=(val_dataset.shuffle(1000).batch(64))

In [ ]:
# tf.constant(list(train_dataset.as_numpy_iterator())[7])

#U-Net Model

In [7]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
import numpy as np

# from data import channels
# from data import image_cols
# from data import image_rows
# from data import modalities

batch_norm = True
smooth = 1.0
image_rows=256
image_cols=256 
channels=1
modalities=3

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


def unet():
    inputs = Input((image_rows, image_cols, channels * modalities))

    conv1 = Conv2D(32, (3, 3), padding='same',data_format="channels_last")(inputs)
    if batch_norm:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)

    conv1 = Conv2D(32, (3, 3), padding='same',data_format="channels_last")(conv1)
    if batch_norm:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2), padding='same',data_format="channels_last")(conv1)

    conv2 = Conv2D(64, (3, 3), padding='same',data_format="channels_last")(pool1)
    if batch_norm:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation('relu')(conv2)

    conv2 = Conv2D(64, (3, 3), padding='same',data_format="channels_last")(conv2)
    if batch_norm:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation('relu')(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2), padding='same',data_format="channels_last")(conv2)

    conv3 = Conv2D(128, (3, 3), padding='same',data_format="channels_last")(pool2)
    if batch_norm:
        conv3 = BatchNormalization(axis=3)(conv3)
    conv3 = Activation('relu')(conv3)

    conv3 = Conv2D(128, (3, 3), padding='same',data_format="channels_last")(conv3)
    if batch_norm:
        conv3 = BatchNormalization(axis=3)(conv3)
    conv3 = Activation('relu')(conv3)

    pool3 = MaxPooling2D(pool_size=(2, 2), padding='same',data_format="channels_last")(conv3)

    conv4 = Conv2D(256, (3, 3), padding='same',data_format="channels_last")(pool3)
    if batch_norm:
        conv4 = BatchNormalization(axis=3)(conv4)
    conv4 = Activation('relu')(conv4)

    conv4 = Conv2D(256, (3, 3), padding='same',data_format="channels_last")(conv4)
    if batch_norm:
        conv4 = BatchNormalization(axis=3)(conv4)
    conv4 = Activation('relu')(conv4)

    pool4 = MaxPooling2D(pool_size=(2, 2), padding='same',data_format="channels_last")(conv4)

    conv5 = Conv2D(512, (3, 3), padding='same',data_format="channels_last")(pool4)
    if batch_norm:
        conv5 = BatchNormalization(axis=3)(conv5)
    conv5 = Activation('relu')(conv5)

    conv5 = Conv2D(512, (3, 3), padding='same',data_format="channels_last")(conv5)
    if batch_norm: 
        conv5 = BatchNormalization(axis=3)(conv5)
    conv5 = Activation('relu')(conv5)

    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv4], axis=3)

    conv6 = Conv2D(256, (3, 3), padding='same')(up6)
    if batch_norm:
        conv6 = BatchNormalization(axis=3)(conv6)
    conv6 = Activation('relu')(conv6)

    conv6 = Conv2D(256, (3, 3), padding='same')(conv6)
    if batch_norm:
        conv6 = BatchNormalization(axis=3)(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3], axis=3)

    conv7 = Conv2D(128, (3, 3), padding='same')(up7)
    if batch_norm:
        conv7 = BatchNormalization(axis=3)(conv7)
    conv7 = Activation('relu')(conv7)

    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    if batch_norm:
        conv7 = BatchNormalization(axis=3)(conv7)
    conv7 = Activation('relu')(conv7)

    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2], axis=3)

    conv8 = Conv2D(64, (3, 3), padding='same')(up8)
    if batch_norm:
        conv8 = BatchNormalization(axis=3)(conv8)
    conv8 = Activation('relu')(conv8)

    conv8 = Conv2D(64, (3, 3), padding='same')(conv8)
    if batch_norm:
        conv8 = BatchNormalization(axis=3)(conv8)
    conv8 = Activation('relu')(conv8)

    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1], axis=3)

    conv9 = Conv2D(32, (3, 3), padding='same')(up9)
    if batch_norm:
        conv9 = BatchNormalization(axis=3)(conv9)
    conv9 = Activation('relu')(conv9)

    conv9 = Conv2D(32, (3, 3), padding='same')(conv9)
    if batch_norm:
        conv9 = BatchNormalization(axis=3)(conv9)
    conv9 = Activation('relu')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    return model

#Transfer Learning

In [ ]:
# from importlib import reload  # Python 3.4+
# import net
# reload(net)

<module 'net' from '/content/net.py'>

In [8]:
from tensorflow.keras.models import  Model
# from net import unet,dice_coef_loss,dice_coef,iou
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [9]:
model=unet()
# model.load_weights("/content/drive/MyDrive/MscProj/models/unet/weights_64.h5")

In [ ]:
model.summary()

In [11]:
#get the old config
old_config=model.get_config()

h,w,c=256,256,1
#update the old config
old_config['layers'][0]['config']['batch_input_shape']=(None, h, w, c)

model_up=Model.from_config(old_config)


#update the weights
def avg_weights(weights):
    average_weights=np.mean(weights,axis=-2).reshape(weights[:,:,-1:,:].shape)
    return average_weights
new_config=model_up.get_config()
new_config_layer_names=[new_config['layers'][x]['name'] for x in range(len(new_config['layers']))]
first_conv_name=new_config_layer_names[1]
for layer in model.layers:
    if(layer.name in new_config_layer_names):
        if(layer.get_weights()!=[]):
            target_layer=model_up.get_layer(layer.name)
            if (layer.name==first_conv_name):
                weights=layer.get_weights()[0]
                biases=layer.get_weights()[1]
                weights_single=avg_weights(weights)
                target_layer.set_weights([weights_single,biases])
            else:
                target_layer.set_weights(layer.get_weights())

In [12]:
# Metrics
lr=0.001
metrics = [dice_coef, iou, Recall(), Precision()]

model_up.compile(loss=dice_coef_loss, optimizer=Adam(lr), metrics=metrics)

In [ ]:
# model_up.predict(list(train_dataset.as_numpy_iterator())[0])
list(train_dataset.as_numpy_iterator())[1][0][1].shape
model_up.predict(list(train_dataset.as_numpy_iterator())[1][0])[1]

In [13]:

model_up.fit(
        train_batches,
        epochs=10,
        validation_data=val_batches,
        callbacks=[TensorBoard("./graphs")],
        shuffle=False
    )

Epoch 1/10
9/9 [==============================] - 62s 4s/step - loss: 0.9167 - dice_coef: 0.0838 - iou: 0.0439 - recall: 0.8745 - precision: 0.0526 - val_loss: 0.9773 - val_dice_coef: 0.0227 - val_iou: 0.0115 - val_recall: 1.0000 - val_precision: 0.0110
Epoch 2/10
9/9 [==============================] - 13s 1s/step - loss: 0.8738 - dice_coef: 0.1269 - iou: 0.0679 - recall: 0.9341 - precision: 0.1116 - val_loss: 0.9761 - val_dice_coef: 0.0238 - val_iou: 0.0121 - val_recall: 0.3691 - val_precision: 0.0151
Epoch 3/10
9/9 [==============================] - 13s 1s/step - loss: 0.8241 - dice_coef: 0.1761 - iou: 0.0966 - recall: 0.9464 - precision: 0.3162 - val_loss: 0.9851 - val_dice_coef: 0.0148 - val_iou: 0.0075 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 13s 1s/step - loss: 0.7802 - dice_coef: 0.2208 - iou: 0.1243 - recall: 0.9558 - precision: 0.4965 - val_loss: 0.9765 - val_dice_coef: 0.0236 - val_iou: 0.0119 - val_recall: 0.9680 

In [ ]:
model_up.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 32  320         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 activation_18 (Activation)     (None, 256, 256, 32  0           ['conv2d_19[0][0]']              
                                )                                                           

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "/content/graphs/train"

##Model testing